In [1]:
import time
import calendar
import pandas as pd
import numpy as np

CITY_DATA = { 'chicago': 'chicago.csv',
              'new york city': 'new_york_city.csv',
              'washington': 'washington.csv' }

In [2]:
def get_filters():
    
    """
    Asks user to specify a city, month, and day to analyze.

    Returns:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of week to filter by, or "all" to apply no day filter
    """
    print('Hello! Let\'s explore some US bikeshare data!')
    # get user input for city (chicago, new york city, washington). HINT: Use a while loop to handle invalid inputs
    while True:
            
        try:
            city = input("Would you like to see data for Chicago, New York, or Washington?")
        except Exception as e:
            print(e + " Please try again.")
            #Return to the start of the the loop 
            continue
        
        if city.lower() not in ('chicago', 'new york', 'washington'):
            print(city + " is not a valid city. Please try again")
            continue
        
        else:
            #city was successfully parsed!
            #We're ready to exit the loop.
            break

    city = city.lower()

    
#    while True:
        
#        try:
#            filterBy = input("Would you like to filter the data by month, day, both, or not at all? Type ""none"" for no time filter.")
#
#        except Exception as e:
#            print(e + " Please try again.")
#            #Return to the start of the the loop 
#            continue
        
        
#        if filterBy.lower() not in ('month', 'day', 'both', 'none'):
#            print(filterBy + " is not a valid filter. Please try again")
#            continue
#        else:
            #filter was successfully parsed!
            #We're ready to exit the loop.
#           break
    
#    filterBy = filterBy.lower()


    
#    while filterBy in ('none'):
#        month = "all"
#        day = "all"
#        break

        
    # get user input for month (all, january, february, ... , june)


#    while filterBy in ('month', 'both'):
    while True:        
        try:
            month = input("Which month? Jan, Feb, Mar, Apr, May, or Jun?")
        except Exception as e:
            print(e + " Please try again.")
            #Return to the start of the the loop 
            continue
            
        if month.lower() not in ('jan', 'feb', 'mar', 'apr', 'may', 'jun'):
            print(month + " is not a valid month. Please try again")
            continue
        else:
            #month was successfully parsed!
            #We're ready to exit the loop.
            break           

    month = month.lower()        



    # get user input for day of week (all, monday, tuesday, ... sunday)

#    while filterBy in ('day', 'both'):       
    while True:        

        try:
            day = input("Which day? Please type your response as an integer (e.g., 1=Sunday).")
        except Exception as e:
            print(e + " Please try again.")
            #Return to the start of the the loop 
            continue
            
        if 7 > int(day) < 0:
            print(day + " is not a valid day. Please try again")
            continue
        else:
            #month was successfully parsed!
            #We're ready to exit the loop.
            break   
    
    day = day

    print('-'*40)
    return city, month, day

In [3]:
def load_data(city, month, day):    
    """
    Loads data for the specified city and filters by month and day if applicable.

    Args:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of week to filter by, or "all" to apply no day filter
    Returns:
        df - Pandas DataFrame containing city data filtered by month and day
    """ 
    
    # load data file into a dataframe
    df = pd.read_csv(CITY_DATA[city])

    # convert the Start Time column to datetime
    df['Start Time'] = pd.to_datetime(df['Start Time'])

    # extract month and day of week from Start Time to create new columns
    df['month'] = df['Start Time'].dt.month
    df['day_of_week'] = df['Start Time'].dt.weekday_name
    # extract hour from the Start Time column to create an hour column
    df['hour'] = df['Start Time'].dt.hour

    # filter by month if applicable
    if month != 'all':
        # use the index of the months list to get the corresponding int
        months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun']
        month = months.index(month) + 1

        # filter by month to create the new dataframe
        df = df[df['month'] == month]
        
    # filter by day of week if applicable
    if day != 'all':
        days = pd.Series(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"])
        day = days.iloc[int(day) - 1]
        # filter by day of week to create the new dataframe
        df = df[df['day_of_week'] == day.title()] 
        
    return df

In [4]:
def time_stats(df):
    """Displays statistics on the most frequent times of travel."""

    print('\nCalculating The Most Frequent Times of Travel...\n')
    start_time = time.time()

    # display the most common month    
    common_month = calendar.month_name[df['month'].mode()[0]]
    print("What is the most popular day for traveling?")
    print(common_month)
    # wait for 1 seconds
    time.sleep(1)

    # display the most common day of week
    common_day = df['day_of_week'].mode()[0]
    print("\nWhat is the most popular day for traveling?")
    print(common_day)
    # wait for 1 seconds
    time.sleep(1)

    # display the most common start hour
    common_hour = df['hour'].mode()[0]
    print("\nWhat is the most popular hour of the day to start your travels?")
    print(common_hour) 
    # wait for 1 seconds
    time.sleep(1)

    print("\nThis took %s seconds." % (time.time() - start_time))    
    print('-'*40) 

In [5]:
def station_stats(df):
    """Displays statistics on the most popular stations and trip."""

    print('\nCalculating The Most Popular Stations and Trip...\n')
    start_time = time.time()

    # display most commonly used start station
    common_start_station = df['Start Station'].mode()[0]
    print("Which station is the most popular to start your travel?")
    print(common_start_station)
    # wait for 1 seconds
    time.sleep(1)

    # display most commonly used end station
    common_end_station = df['End Station'].mode()[0]
    print("\nWhich  station is the most popular to end your travel?")
    print(common_end_station)
    # wait for 1 seconds
    time.sleep(1)

    # display most frequent combination of start station and end station trip


    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

In [6]:
def convert_seconds(seconds):
    days, seconds = divmod(seconds, 24 * 60 * 60)
    hours, seconds = divmod(seconds, 60 * 60)
    minutes, seconds = divmod(seconds, 60)
    return days, hours, minutes, seconds

In [7]:
def trip_duration_stats(df):
    """Displays statistics on the total and average trip duration."""

    print('\nCalculating Trip Duration...\n')
    start_time = time.time()

    # display total travel time
    total_travel_time = df['Trip Duration'].sum()
    
    total_days, total_hrs, total_mins, total_seconds = convert_seconds(total_travel_time)
    print("What was the total traveling done for 2017 through {}?".format(calendar.month_name[df['month'].mode()[0]]))
    print(total_days, 'days', total_hrs, 'hours', total_mins, 'minutes', total_seconds, 'seconds')
    # wait for 1 seconds
    time.sleep(1)

    # display mean travel time
    mean_travel_time = int(df['Trip Duration'].mean())
    mean_days, mean_hrs, mean_mins, mean_seconds = convert_seconds(mean_travel_time)
    print("\nWhat was the average time spent on each trip?")
    print(mean_days, 'days', mean_hrs, 'hours', mean_mins, 'minutes', mean_seconds, 'seconds')
    # wait for 1 seconds
    time.sleep(1)

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

In [8]:
def user_stats(df):
    """Displays statistics on bikeshare users."""

    print('\nCalculating User Stats...\n')
    start_time = time.time()

    # Display counts of user types
    user_types = df['User Type'].value_counts().to_csv(header=None,sep='\t')   
    print("What is the breakdown of users?")
    print(user_types)
    time.sleep(1)

    # Display counts of gender
    gender_types = df['Gender'].value_counts().to_csv(header=None,sep='\t')   
    print("What is the breakdown of gender?")
    print(gender_types)
    time.sleep(1)

    # Display earliest, most recent, and most common year of birth
    earliest_birth = int(df['Birth Year'].min())
    recent_birth = int(df['Birth Year'].max())
    common_birth = int(df['Birth Year'].mode()[0])
    print("What is the oldest, youngest, and most popular year of birth, respectively?")
    print('Oldest year:', earliest_birth)
    print('Youngest year:', recent_birth)
    print('Popular year:', common_birth)
    time.sleep(1)

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

In [9]:
def main():
    while True:
        city, month, day = get_filters()

        df = load_data(city, month, day)

        time_stats(df)
        station_stats(df)
        trip_duration_stats(df)
        user_stats(df)

        restart = input('\nWould you like to restart? Enter yes or no.\n')
        if restart.lower() != 'yes':
            break



if __name__ == "__main__":
	main()


Hello! Let's explore some US bikeshare data!
Would you like to see data for Chicago, New York, or Washington?chicago
Which month? Jan, Feb, Mar, Apr, May, or Jun?jan
Which day? Please type your response as an integer (e.g., 1=Sunday).1
----------------------------------------

Calculating The Most Frequent Times of Travel...

What is the most popular day for traveling?
January

What is the most popular day for traveling?
Sunday

What is the most popular hour of the day to start your travels?
13

This took 3.0089991092681885 seconds.
----------------------------------------

Calculating The Most Popular Stations and Trip...

Which station is the most popular to start your travel?
McClurg Ct & Illinois St

Which  station is the most popular to end your travel?
Wabash Ave & Roosevelt Rd

This took 2.0214993953704834 seconds.
----------------------------------------

Calculating Trip Duration...

What was the total traveling done for 2017 through January?
15 days 21 hours 23 minutes 57 sec